# Install SlayerPytorch on Colab
After the installations the runtime needs to be restarted. 
```
exit()
```
Will restart the runtime without deleting files. The runtime will automatically start. And if you press "run all" the run is not interrupted and works till the end.

In [ ]:
!git clone https://github.com/bamsumit/slayerPytorch
!pip install ninja
exit()

In [ ]:
%cd slayerPytorch/
!python setup.py install
exit()

Test to verify if everything went well with the installation

In [ ]:
%cd slayerPytorch/test/
!python -m  unittest

# Extract NMNISTsmall dataset
This is a subset of NMNIST dataset containing first 1000 training samples and first 100 testing samples. The original NMNSIT dataset consists of full MNIST samples converted into spikes using DVS sensor moved in three repeatable saccadic motion. For details and full dataset download links, refer to [https://www.garrickorchard.com/datasets/n-mnist](https://www.garrickorchard.com/datasets/n-mnist)

In [ ]:
!unzip -q '/content/drive/MyDrive/NMNIST.zip' -d '/'

# SNN Configuration

In [ ]:
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../../src")

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import slayerSNN as snn
import zipfile
import shutil

# Read SNN configuration from yaml file
netParams = snn.params('/content/slayerPytorch/exampleLoihi/02_NMNIST/network.yaml')

# Ts   = netParams['simulation']['Ts']
# Ns   = int(netParams['simulation']['tSample'] / netParams['simulation']['Ts'])
# Nin  = int(netParams['layer'][0]['dim'])
# Nhid = int(netParams['layer'][1]['dim'])
# Nout = int(netParams['layer'][2]['dim'])

			
# Define dataset module
class nmnistDataset(Dataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength):
        self.path = datasetPath 
        self.samples = np.loadtxt(sampleFile).astype('int')
        self.samplingTime = samplingTime
        self.nTimeBins    = int(sampleLength / samplingTime)

    def __getitem__(self, index):
        # Read inoput and label
        inputIndex  = self.samples[index, 0]
        classLabel  = self.samples[index, 1]
        # Read input spike
        inputSpikes = snn.io.readNpSpikes(
                        self.path + str(inputIndex.item()) + '.npy'
                        ).toSpikeTensor(torch.zeros((2,34,34,self.nTimeBins)),
                        samplingTime=self.samplingTime)
        # Create one-hot encoded desired matrix
        desiredClass = torch.zeros((10, 1, 1, 1))
        desiredClass[classLabel,...] = 1
        # Input spikes are reshaped to ignore the spatial dimension and the neurons are placed in channel dimension.
        # The spatial dimension can be maintained and used as it is.
        # It requires different definition of the dense layer.
        return inputSpikes, desiredClass, classLabel
        #return inputSpikes.reshape((-1, 1, 1, inputSpikes.shape[-1])), desiredClass, classLabel

    def __len__(self):
        return self.samples.shape[0]

# Define the network
class Network(torch.nn.Module):
    def __init__(self, netParams):
        super(Network, self).__init__()
        # initialize slayer
        slayer = snn.loihi(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        # define network functions
        self.fc1   = slayer.dense((34*34*2), 512)
        self.fc2   = slayer.dense(512, 10)

    def forward(self, spikeInput):
        spike = self.slayer.spikeLoihi(self.fc1(spikeInput))
        spike = self.slayer.delayShift(spike, 1)
        # A minimum axonal delay of 1 tick is required in Loihi hardare
        spike = self.slayer.spikeLoihi(self.fc2(spike))
        spike = self.slayer.delayShift(spike, 1)
        return spike

def save_ckp(state, is_best_loss, is_best_acc, checkpoint_dir, best_model_dir):
    f_path = checkpoint_dir+'checkmnist.pt'
    torch.save(state, f_path)
    if is_best_loss:
        best_fpath = best_model_dir+'bestmnist.pt'
        shutil.copyfile(f_path, best_fpath)
    if is_best_acc:
        acc_fpath =  best_model_dir+'best_acc_mnist.pt'
        shutil.copyfile(f_path, acc_fpath)
        
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

if __name__ == '__main__':
	# Define the cuda device to run the code on.
	device = torch.device('cuda')

	# Create network instance.
	net = Network(netParams).to(device)

	# Create snn loss instance.
	error = snn.loss(netParams, snn.loihi).to(device)

	# Define optimizer module.
	optimizer = snn.utils.optim.Nadam(net.parameters(), lr = 0.01, amsgrad = True)
 
  # Learning stats instance.
  stats = snn.utils.stats()
      
  ckp_path = "/content/drive/My Drive/checkpoint/best_acc_mnist.pt"
  net, optimizer, start_epoch = load_ckp(ckp_path, net, optimizer)  


	

#Adversarial Attacks
Collection of 5 different adversarial attacks



##Gradient Based Attack

In [ ]:
testingSet = nmnistDataset(datasetPath  =  '/content/NMNIST/Test/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
testLoader = DataLoader(dataset=testingSet, batch_size=1, shuffle=False, num_workers=4)

import argparse
import numpy as np
import pdb 
import torch, torchvision
from torch.utils.data import DataLoader
from torch.autograd import Variable
#from dataset import *
#from models import *

def update(x,y,t,s,A):
  for i in range(x-s,x+s+1):
     for j in range(y-s,y+s+1):
        if not(i==x and j==y) and i>0 and j>0 and i<34 and j<34:
	        A[i][j] = t
    

#Select the device to run the code 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ('Device: ' + str(device))

def calc_gradients(
		test_dataloader,
		model,
		max_iter,
		learning_rate,
    samples,
		filter_on,
		s,
		T,
		targets=None,
		weight_loss2=1,
		batch_size=1,
		seq_len=40,):
	
	#Define the modifier and the optimizer

  #modif = torch.Tensor(1, seq_len, 2, 34, 34).fill_(1).to(device)
  #modifier = torch.nn.Parameter(modif, requires_grad=True)
  #optimizer = torch.optim.Adam([modifier], lr=learning_rate)
  min_loss = 1e-5
  prev_loss = 1e-5
  correct = np.zeros(max_iter)	
  for batch_index, (input_image, input_target, input_label) in enumerate(test_dataloader):

    modif = torch.Tensor(1, seq_len, 2, 34, 34).fill_(1).to(device)
    modifier = torch.nn.Parameter(modif, requires_grad=True)
    optimizer = torch.optim.Adam([modifier], lr=learning_rate)


    #if batch_index>samples:
     # continue
    model.eval()
    with torch.no_grad():	
      input_image, input_label = input_image.to(device), input_label.to(device)

    #Clean video prediction 
    print(f'Batch Number: {batch_index}/{len(test_dataloader)}')
    print('------------------prediction for clean video-------------------')
    input_image = Variable(input_image, requires_grad=True)
    output = model.forward(torch.reshape(input_image, (1,2312,1,1,350)))
    pre_label= snn.predict.getClass(output)
    p = torch.nn.Softmax(dim=1)		
    #numSpikes = torch.sum(output, 4, keepdim=True).cpu()	
    #probs= p(numSpikes.reshape((numSpikes.shape[0], -1)))
    print (f'Prediction: {pre_label}, Original_label: {input_label.cpu().numpy()}')

    print('------------------prediction for adversarial video-------------------')

    min_in = input_image.min().detach() #0
    max_in = input_image.max().detach() #1
    
    all_loss = []
    
    for iiter in range(max_iter):
      with open ('/content/modifier.txt','w') as file:
        file.write(str(modifier))      
      print(modifier[0,0,0,0,0])			
      input_image = Variable(input_image, requires_grad=True)
      #model.lstm.reset_hidden_state()
      
      #Frames to be perturbed
      indicator = [0]*350
      for i,x in enumerate(indicator):
        if (i>95 and i<125) or (i>175 and i<210) or (i>290 and i<310 ):
          x=1
    
      #Perturbating the frames
      #print(modifier.shape, input_image.shape)
      input_image=torch.reshape(input_image, (1,350,2,34,34))
      #print("input image shape:{}".format(input_image.shape))
      true_image = torch.clamp((modifier[0,0,:,:,:]+input_image[0,0,:,:,:]), min_in, max_in)
      true_image = torch.unsqueeze(true_image, 0)
      
      for ll in range(seq_len-1): #seq_len =1450
        if indicator[ll+1] == 1:
          mask_temp = torch.clamp((modifier[0,ll+1,:,:,:]+input_image[0,ll+1,:,:,:]), min_in, max_in)
        else:
          mask_temp = input_image[0,ll+1,:,:,:]
        mask_temp = torch.unsqueeze(mask_temp,0)
        true_image = torch.cat((true_image, mask_temp),0)
      #true_image = torch.unsqueeze(true_image, 0)

      #######		 Filter section 	######
      if filter_on :
        img = torch.reshape(true_image, (2,34,34,350)).cpu().detach()
        TD = snn.io.spikeArrayToEvent(img.numpy(), samplingTime=1)
        snn.io.encodeNpSpikes("/content/drive/My Drive/temp/img.npy", TD)
        data= np.load('/content/drive/My Drive/temp/img.npy'.format(i))
        data= data[data[:,3].argsort()] #sort elements by timestamp
        temp=np.zeros((34,34))
        real=[]
        for d in data:
          update(int(d[0]),int(d[1]),d[3],s,temp)
          if d[3]-temp[int(d[0])][int(d[1])]<T:
            real.append(d)
        real=np.stack(real, axis=0 )
        with open("/content/drive/My Drive/temp/fil.npy", "wb") as f:
          np.save(f,real)
        true_image = snn.io.readNpSpikes(
            "/content/drive/My Drive/temp/fil.npy"
            ).toSpikeTensor(torch.zeros((2,34,34,350)),
            samplingTime=1.0)
        true_image =  true_image.to(device)
      true_image = torch.reshape(true_image, (1,350,2,34,34))			
      #######		 End Filter Section 	 #######							
     
      #Prediction on the adversarial video
      output = model.forward(torch.reshape(true_image, (1,2312,1,1,350)))
      pre_label = snn.predict.getClass(output)
      numSpikes = torch.sum(output, 4, keepdim=True)
      numSpikes= torch.div(numSpikes,torch.max(numSpikes))	
      probs= p(numSpikes.reshape((numSpikes.shape[0], -1)))	 
      #extracting the probability of true label 
      zero_array = torch.zeros(10).to(device) ## 1450 invece di 11 e probs=p(output)
      zero_array[input_label.cpu()] = 1
      true_label_onehot = probs*zero_array
      true_label_prob = torch.sum(true_label_onehot, 1)
      #print (f'Prediction: {pre_label}, Original_label: {input_label.cpu().numpy()}')
      if pre_label.numpy()==input_label.cpu().numpy() : correct[iiter]+=1
      #print("""True label prob : {}\n zero_array[input_label.cpu()]: {} \n input_label.cpu(): {}""".format(true_label_prob,zero_array[input_label.cpu()],input_label.cpu() ))		
      #Loss
      if targets is None:
        loss1 = -torch.log(1 - true_label_prob + 1e-6)
      else:
        loss1 = -torch.log(true_label_prob + 1e-6)
      loss1 = torch.mean(loss1)
      input_image=torch.reshape(input_image, (1,350,2,34,34))
      loss2 = torch.sum(torch.sqrt(torch.mean(torch.pow((true_image-input_image+1e-6), 2), dim=0).mean(dim=2).mean(dim=2).mean(dim=1)))

      #norm_frame = torch.mean(torch.abs(modifier), dim=3).mean(dim=3).mean(dim=2) 

      loss = loss1 + weight_loss2 * loss2

      optimizer.zero_grad()
      loss.backward() # check retain_graph loss.backward(retain_graph=True)
      optimizer.step()

      if True: #iiter % max_iter == 0: 
        print (f'Probability for ground truth label : {true_label_prob.detach().cpu().numpy()}')
        if prev_loss < loss : 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}(\u25b2), Loss1: {loss1}, Loss2: {loss2}')
        elif prev_loss > loss: 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}(\u25bc), Loss1: {loss1}, Loss2: {loss2}')
        else: 
          print(f'Iteration: [{iiter}/{max_iter}], Loss: {loss}, Loss1: {loss1}, Loss2: {loss2}')
      prev_loss = loss

      break_condition = False
      if loss < min_loss:
        if torch.abs(loss-min_loss) < 0.0001:
            break_condition = True
            print ('Aborting early!')
        min_loss = loss

      # Empty cache
      if torch.cuda.is_available():
        torch.cuda.empty_cache()
      #print(numSpikes,totSpikes, totSpikes.shape, numSpikes.shape)
    # Save adversarial dataset as spike events
    image = torch.reshape(true_image, (2,34,34,350)).cpu().detach()
    TD = snn.io.spikeArrayToEvent(image.numpy(), samplingTime=1)
    snn.io.encodeNpSpikes("/content/prova/{}.npy".format(batch_index), TD)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()	
  print(correct)


def main(net):

  args = {
      'description': 'Sparse Adversarial Perturbations',
      'input_dir' : '/content/drive/My Drive/Test/',
      'num_iter': 20,
      'learning_rate': 100,
      'samples':11,
			'filter_on': False,
			's': 2,
			'T': 5,
      'target': None,
      'weight_loss2': 0,
      'split_path': None,
      'split_number': 1,
      'img_dim': 34,
      'channels': 2,
      'checkpoint_model': '../../../drive/My Drive/best/check.pt',
			}
  seq_len = 350 #number of frames in a video
  batch_size = 1
  targets = None
  image_shape = (args['channels'], args['img_dim'], args['img_dim'])
  net = net.to(device)
  net.train()
  print('Model Loaded Successfully!')
	#Call the function to generate the adversarial videos
  calc_gradients(
	testLoader,
	net,
	args['num_iter'],
	args['learning_rate'],
  args['samples'],
	args['filter_on'],
	args['s'],
	args['T'],
	targets,
	args['weight_loss2'],
	batch_size,
	seq_len,
	)

if __name__ == '__main__':
	main(net)

Output streaming troncato alle ultime 5000 righe.
tensor(1.2486, device='cuda:0', grad_fn=<SelectBackward>)
Probability for ground truth label : [0.10794517]
Iteration: [18/20], Loss: 0.11422651261091232(▲), Loss1: 0.11422651261091232, Loss2: 0.8917696475982666
tensor(1.2590, device='cuda:0', grad_fn=<SelectBackward>)
Probability for ground truth label : [0.10794517]
Iteration: [19/20], Loss: 0.11422651261091232, Loss1: 0.11422651261091232, Loss2: 0.8922889232635498
Batch Number: 22/100
------------------prediction for clean video-------------------
Prediction: tensor([6]), Original_label: [6]
------------------prediction for adversarial video-------------------
tensor(1., device='cuda:0', grad_fn=<SelectBackward>)
Probability for ground truth label : [0.09702757]
Iteration: [0/20], Loss: 0.10206209868192673(▼), Loss1: 0.10206209868192673, Loss2: 1.0003498792648315
tensor(0.9364, device='cuda:0', grad_fn=<SelectBackward>)
Probability for ground truth label : [0.09155758]
Iteration: [1/

##Frame Attack

In [ ]:
import numpy as np
import torch
import time

def cornerGen(x=0,y=0, N=34,T=350):
  corner = torch.zeros((N,N,T))
  for i in range(N):
    for j in range(N):
      if i==x or i == N-1-x or j==y or j== N-1-y :
        corner[i,j,:]=1       

  return corner.expand(2,N,N,T)


samples = [i for i in range(100)]
stats.testing.reset()

N=34
x=0
y=0

start=time.time()

temp= list(samples)
corner= cornerGen(x,y)
for i  in temp:

  input,target,label=testSet[i]
  
  adv= input.unsqueeze(dim=0)+corner
  
  # Save adversarial dataset as spike events
  image = torch.reshape(adv, (2,N,N,350))
  TD = snn.io.spikeArrayToEvent(image.numpy(), samplingTime=1)
  snn.io.encodeNpSpikes("/content/adversarial/{}.npy".format(i), TD)
print(time.time()-start)

## Corner, Dash, MF-Aware Dash Attacks

In [ ]:
import numpy as np
import torch
import time

def cornerGen(x=0,y=2,left=False, N=34,T=350):
  dash = torch.zeros((2,N,N,T))
  for i in range(N):
    for j in range(N):
      if left:
        if (i==x and (j<y)):
          dash[:,i,j,:]=1
      else:
        if (i==x and (j<y)):
          dash[:,i,j,:]=1
  return dash  

def dashGen(x=0,y=2,left=False, N=34,T=350):
  dash = torch.zeros((2,N,N,T))
  for i in range(N):
    for j in range(N):
      if left:
        if (i==x and (j==y or j== y-1)):
          dash[:,i,j,:]=1
      else:
        if (i==x and (j==N-y or j== N-y-1)):
          dash[:,i,j,:]=1
  return dash 

def MFAwareDashGen(x=0,y=2,left=False,th0=10, N=34,T=350,):
  th=th0
  dash = torch.zeros((2,N,N,T))
  for t in range(T):
    for i in range(N):
      for j in range(N):
        if left:
          if (i==x and (j==y or j== y-1) and t<th):
            dash[1,i,j,t]=1
        else:
          if (i==x and (j==N-y or j== N-y-1) and t<th):
            dash[0,i,j,t]=1
    if t==th:
      th+=th0
      y+=2

  return dash

testSet = nmnistDataset(datasetPath  =  '/content/NMNIST/Test/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
testLoader = DataLoader(dataset=testSet, batch_size=1, shuffle=False, num_workers=4)


samples = [i for i in range(100)]
stats.testing.reset()

N=34
flag=0
left = 1
min_x=0
x=min_x
y=2
th0=50
start=time.time()
while samples and min_x<N/8:
  temp= list(samples)
  corner= MFAwareDashGen(x,y,left,th0) ## select here the desired attack
  for i  in temp:

    input,target,label=testSet[i]
    
    adv= input.unsqueeze(dim=0)+corner
    with torch.no_grad():
      adv  = adv.to(device)
      target = target.to(device) 

    output = net.forward(torch.reshape(adv, (1,2312,1,1,350)))
    if (snn.predict.getClass(output) != label):
      try: 
        samples.remove(i)
      except:
        print(f'{i} not in list')
      # Save adversarial dataset as spike events
      image = torch.reshape(adv, (2,N,N,350)).cpu().detach()
      TD = snn.io.spikeArrayToEvent(image.numpy(), samplingTime=1)
      snn.io.encodeNpSpikes("/content/adversarial/{}.npy".format(i), TD)
  print(f'remaining samples {len(samples)}, x = {x}, y={y}', 'left' if left else 'right')

  if flag:
    x= min_x
    flag^=1
    left^=1
    if left:
      y+=1
  else:
    x= N-1-min_x
    flag^=1
  
  if y>N/2:
    y=2
    min_x+=1
    flag=0
    left = 1


print(time.time()-start)


#Test 
Choose the desired Dataset (Normal, Attacked, Filtered) to verify the results.

In [ ]:
testSet = IBMGestureDataset(datasetPath  =  '/content/Test/', 
                              sampleFile  ='/content/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=1450)
testLoader = DataLoader(dataset=filSet, batch_size=1, shuffle=False, num_workers=4)

for epoch in range(1):
    stats.testing.reset()
    tSt = datetime.now()
    # Testing loop.
    for i, (input, target, label) in enumerate(testLoader, 0):
      net.eval()
      with torch.no_grad():
        input  = input.to(device)
        target = target.to(device) 
    
      output = net.forward(input)
      stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
      stats.testing.numSamples     += len(label)

      loss = error.numSpikes(output, target)
      stats.testing.lossSum += loss.cpu().data.item()
      stats.print(epoch, i)


#  *FILTERS*







## Background Activity filter: analysis on clean input

In [ ]:
import numpy as np

def update(x,y,t,A,s):
  for i in range(x-s,x+s+1):
     for j in range(y-s,y+s+1):
        if not(i==x and j==y) and i>0 and j>0 and i<34 and j<34:
	        A[i][j] = t
testSet = nmnistDataset(datasetPath  =  '/content/NMNIST/Test/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
testLoader = DataLoader(dataset=testSet, batch_size=1, shuffle=False, num_workers=4)
       
for s in [1,2,3,4]:
  stats1 = snn.utils.stats()
  stats5 = snn.utils.stats()
  stats10 = snn.utils.stats()
  stats20 = snn.utils.stats()      
  for i in range(100):     
    input,target,label = testSet[i]    
    data= np.load('/content/NMNIST/Test/{}.npy'.format(i))
    data= data[data[:,3].argsort()] #sort elements by timestamp
    temp=np.zeros((34,34))
    real1=[]
    real5=[]
    real10=[]
    real20=[]
    for d in data:
      update(int(d[0]),int(d[1]),d[3],temp,s)
      if d[3]-temp[int(d[0])][int(d[1])]<2:
        real1.append(d)

      if d[3]-temp[int(d[0])][int(d[1])]<8:
        real5.append(d)   

      if d[3]-temp[int(d[0])][int(d[1])]<15:
        real10.append(d)

      if d[3]-temp[int(d[0])][int(d[1])]<25:
        real20.append(d)  

    real1=np.stack(real1, axis=0 )
    with open("/content/filtro/1.npy", "wb") as f:
      np.save(f,real1)
    input1 = snn.io.readNpSpikes(
        "/content/filtro/1.npy").toSpikeTensor(torch.zeros((2,34,34,350)), samplingTime=1.0)
    input1 =  input1.to(device)
    output1 = net.forward(torch.reshape(input1, (1,2312,1,1,350)))
    stats1.testing.correctSamples += torch.sum( snn.predict.getClass(output1) == label ).data.item()
    stats1.testing.numSamples     += 1
    #stats1.print(1,i)

    real5=np.stack(real5, axis=0 )
    with open("/content/filtro/5.npy", "wb") as f:
      np.save(f,real5)
    input5 = snn.io.readNpSpikes(
        "/content/filtro/5.npy").toSpikeTensor(torch.zeros((2,34,34,350)), samplingTime=1.0)
    input5 =  input5.to(device)
    output5 = net.forward(torch.reshape(input5, (1,2312,1,1,350)))
    stats5.testing.correctSamples += torch.sum( snn.predict.getClass(output5) == label ).data.item()
    stats5.testing.numSamples     += 1
    #stats5.print(1,i)

    real10=np.stack(real10, axis=0 )
    with open("/content/filtro/10.npy", "wb") as f:
      np.save(f,real10)
    input10 = snn.io.readNpSpikes(
        "/content/filtro/10.npy").toSpikeTensor(torch.zeros((2,34,34,350)), samplingTime=1.0)
    input10 =  input10.to(device)
    output10 = net.forward(torch.reshape(input10, (1,2312,1,1,350)))
    stats10.testing.correctSamples += torch.sum( snn.predict.getClass(output10) == label ).data.item()
    stats10.testing.numSamples     += 1
    #stats10.print(1,i)    

    real20=np.stack(real20, axis=0 )
    with open("/content/filtro/20.npy", "wb") as f:
      np.save(f,real20)
    input20= snn.io.readNpSpikes(
        "/content/filtro/20.npy").toSpikeTensor(torch.zeros((2,34,34,350)), samplingTime=1.0)
    input20 =  input20.to(device)
    output20 = net.forward(torch.reshape(input20, (1,2312,1,1,350)))
    stats20.testing.correctSamples += torch.sum( snn.predict.getClass(output20) == label ).data.item()
    stats20.testing.numSamples     += 1

  stats1.print(1,i)
  stats5.print(1,i)
  stats10.print(1,i)
  stats20.print(1,i)

##Background Activity filter: analysis on adversarial examples

In [ ]:
advSet = nmnistDataset(datasetPath  =  '/content/prova/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
advLoader = DataLoader(dataset=advSet, batch_size=1, shuffle=False, num_workers=4)
for epoch in range(1):
    stats.testing.reset()
    tSt = datetime.now()
    # Testing loop.
    # Same steps as Training loops except loss backpropagation and weight update.
    for i, (input, target, label) in enumerate(advLoader, 0):
      net.eval()
      with torch.no_grad():
        input  = input.to(device)
        target = target.to(device) 
    
      output = net.forward(torch.reshape(input, (1,2312,1,1,350)))
      stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
      stats.testing.numSamples     += len(label)

      loss = error.numSpikes(output, target)
      stats.testing.lossSum += loss.cpu().data.item()
      #stats.print(epoch, i)
AA = stats.testing.accuracy()
print(AA)

0.04


## Mask Filter: analysis on clean input

In [ ]:
testSet = nmnistDataset(datasetPath  =  '/content/Test/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
testLoader = DataLoader(dataset=testSet, batch_size=1, shuffle=False, num_workers=4)


for t in range (5,60,5):
  for i in range(0,100):
    input, target, label = testSet[i]
    activity= torch.sum(input, dim=3)
    activity= torch.sum(activity, dim=0)
    mask=torch.where((activity>t),0,1)
    mask=mask.expand(2,34,34)
    output= input[:,:,:,:]*mask.unsqueeze(dim=3)
    # Save  as spike events
    try:
      TD = snn.io.spikeArrayToEvent(output.numpy(), samplingTime=1)
    except ValueError:  #raised if `output` is empty.
      print(i)
      TD = snn.io.spikeArrayToEvent(input.numpy(), samplingTime=1)

    snn.io.encodeNpSpikes("/content/filter/{}.npy".format(i), TD)  


  filSet = nmnistDataset(datasetPath  =  '/content/filter/', 
                                sampleFile  ='/content/NMNIST/Test/test.txt',
                                samplingTime=1.0,
                                sampleLength=350)
  filLoader = DataLoader(dataset=filSet, batch_size=1, shuffle=False, num_workers=4)

  for epoch in range(1):
      stats.testing.reset()
      tSt = datetime.now()
      # Testing loop.
      # Same steps as Training loops except loss backpropagation and weight update.
      for i, (input, target, label) in enumerate(filLoader, 0):
        net.eval()
        with torch.no_grad():
          input  = input.to(device)
          target = target.to(device) 
      
        output = net.forward(torch.reshape(input, (1,2312,1,1,350)))
        stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
        stats.testing.numSamples     += len(label)

        loss = error.numSpikes(output, target)
        stats.testing.lossSum += loss.cpu().data.item()
  print(t,stats.testing.accuracy())


## Mask Filter: analysis on adversarial examples

In [ ]:
advSet = nmnistDataset(datasetPath  =  '/content/adversarial/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
advLoader = DataLoader(dataset=advSet, batch_size=1, shuffle=False, num_workers=4)


for t in range (5,60,5):
  for i in range(0,100):
    input, target, label = advSet[i]
    activity= torch.sum(input, dim=3)
    activity= torch.sum(activity, dim=0)
    mask=torch.where((activity>t),0,1)
    mask=mask.expand(2,34,34)
    output= input[:,:,:,:]*mask.unsqueeze(dim=3)
    # Save  as spike events
    try:
      TD = snn.io.spikeArrayToEvent(output.numpy(), samplingTime=1)
    except ValueError:  #raised if `output` is empty.
      print(i)
      TD = snn.io.spikeArrayToEvent(input.numpy(), samplingTime=1)

    snn.io.encodeNpSpikes("/content/filter/{}.npy".format(i), TD)  


  filSet = nmnistDataset(datasetPath  =  '/content/filter/', 
                                sampleFile  ='/content/NMNIST/Test/test.txt',
                                samplingTime=1.0,
                                sampleLength=350)
  filLoader = DataLoader(dataset=filSet, batch_size=1, shuffle=False, num_workers=4)

  for epoch in range(1):
      stats.testing.reset()
      tSt = datetime.now()
      # Testing loop.
      # Same steps as Training loops except loss backpropagation and weight update.
      for i, (input, target, label) in enumerate(filLoader, 0):
        net.eval()
        with torch.no_grad():
          input  = input.to(device)
          target = target.to(device) 
      
        output = net.forward(torch.reshape(input, (1,2312,1,1,350)))
        stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
        stats.testing.numSamples     += len(label)

        loss = error.numSpikes(output, target)
        stats.testing.lossSum += loss.cpu().data.item()
  print(t,stats.testing.accuracy())


# VISUALIZE

In [ ]:
# Visualize the input spikes (first five samples).
testSet = nmnistDataset(datasetPath  =  '/content/NMNIST/Test/', 
                              sampleFile  ='/content/NMNIST/Test/test.txt',
                              samplingTime=1.0,
                              sampleLength=350)
testLoader = DataLoader(dataset=testSet, batch_size=1, shuffle=False, num_workers=4)

from IPython.display import HTML
for i in range(1):
    input, target, label = testSet[i]
    snn.io.showTD(snn.io.spikeArrayToEvent(input.reshape((2, 34, 34, -1)).cpu().data.numpy()))
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 34, 34, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())
  